In [19]:
import pandas as pd
import numpy as np
import datetime as dt

tax_yr = '2021'

summary = f'{tax_yr}_input.xlsx'


df = pd.read_excel(summary, parse_dates = ['Exec Time', 'Exp'])
df['Exec Time'].fillna(method='ffill',inplace = True)

#df['Exec Time'] = pd.to_datetime(df['Exec Time'], format = '%m/%d/%Y %H:%M').dt.date
df['Exec Time'] = df['Exec Time'].dt.date
df['Pos Effect'].replace(['TO CLOSE', 'TO OPEN'],['CLOSE','OPEN'],inplace=True)
df.rename(columns={'Exec Time': 'Trade Date'}, inplace=True)
df.loc[(df['Type'] == 'STOCK') | (df['Type'] == 'ETF'),'Quantity'] = df['Qty']
df.loc[df['Type'].isin(['CALL', 'PUT']),'Quantity'] = df['Qty'] * 100

df.loc[df['Pos Effect'] == 'OPEN','Cost'] = -df['Quantity'] * df['Price']
df.loc[df['Pos Effect'] == 'CLOSE','Proceeds'] = -df['Quantity'] * df['Price']

def optionid(df):
    if (df['Type'] == 'STOCK'):
        return df['Symbol'] + '_STOCK'
    elif (df['Type'] == 'ETF'):
        return df['Symbol'] + '_ETF'
    else:
        return df['Symbol'] + '_' + str(df['Strike']) + '_' + df['Type'] + '_' + df['Exp'].strftime('%Y-%m-%d')
    
df['Option ID'] = df.apply(optionid,axis = 1)
df = df[['Symbol','Option ID', 'Pos Effect','Trade Date', 'Cost','Type', 'Proceeds','Qty','Price']]
df.sort_values(by = ['Symbol','Trade Date'],inplace = True)

In [20]:
df.head()

,Symbol,Option ID,Pos Effect,Trade Date,Cost,Type,Proceeds,Qty,Price
1,AMC,AMC_40.0_CALL_2021-02-19,OPEN,2021-01-28,-580.0,CALL,NaN,4,1.45
0,AMC,AMC_40.0_CALL_2021-02-19,CLOSE,2021-01-29,NaN,CALL,1700.0,-4,4.25
5,AMD,AMD_100.0_CALL_2022-01-21,OPEN,2021-01-27,-3100.0,CALL,NaN,2,15.50
2,AMD,AMD_100.0_CALL_2022-01-21,CLOSE,2021-10-25,NaN,CALL,2400.0,-1,24.00
3,AMD,AMD_100.0_CALL_2022-01-21,CLOSE,2021-10-25,NaN,CALL,2400.0,-1,24.00


In [23]:
df['Cost'].sum() + df['Proceeds'].sum()

8288.98000000004

In [24]:
summary = df.groupby(['Symbol','Option ID','Pos Effect', 'Trade Date']).agg({'Cost':'sum','Proceeds':'sum','Qty':'sum','Price':'mean'})

In [25]:
summary

Cost  Proceeds  \
Symbol Option ID                  Pos Effect Trade Date                     
AMC    AMC_40.0_CALL_2021-02-19   CLOSE      2021-01-29     0.0   1700.00   
                                  OPEN       2021-01-28  -580.0      0.00   
AMD    AMD_100.0_CALL_2022-01-21  CLOSE      2021-10-25     0.0   9600.00   
                                  OPEN       2021-01-27 -3100.0      0.00   
AMZN   AMZN_STOCK                 CLOSE      2021-01-15     0.0   6219.05   
...                                                         ...       ...   
WISH   WISH_11.5_PUT_2021-07-16   CLOSE      2021-07-16     0.0   -440.00   
                                  OPEN       2021-06-21   240.0      0.00   
XPEV   XPEV_100.0_CALL_2022-01-21 CLOSE      2021-10-28     0.0    125.00   
                                  OPEN       2001-11-21 -3350.0      0.00   
       XPEV_STOCK                 CLOSE      2021-01-29     0.0   2510.00   

                                                         Qty     Price  
Symbol Option ID                  Pos Effect Trade Date                 
AMC    AMC_40.0_CALL_2021-02-19   CLOSE      2021-01-29   -4     4.250  
                                  OPEN       2021-01-28    4     1.450  
AMD    AMD_100.0_CALL_2022-01-21  CLOSE      2021-10-25   -4    24.000  
                                  OPEN       2021-01-27    2    15.500  
AMZN   AMZN_STOCK                 CLOSE      2021-01-15   -2  3109.525  
...                                                      ...       ...  
WISH   WISH_11.5_PUT_2021-07-16   CLOSE      2021-07-16    2     2.200  
                                  OPEN       2021-06-21   -2     1.200  
XPEV   XPEV_100.0_CALL_2022-01-21 CLOSE      2021-10-28   -5     0.250  
                                  OPEN       2001-11-21    5     6.700  
       XPEV_STOCK                 CLOSE      2021-01-29  -50    50.200  

[145 rows x 4 columns]

In [26]:
summary['Cost'].sum() + summary['Proceeds'].sum()

8288.98000000004